In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction
In this notebook, we will demonstrate how to copy and paste Page resources within the _SAME_ agent from a `Source` Flow to a `Target` Flow.

These same methods/functions can be further modified to move pages _BETWEEN_ Agents as well!

## Prerequisites
- Ensure you have a GCP Service Account key with the Dialogflow API Admin privileges assigned to it.

In [ ]:
#If you haven't already, make sure you install the `dfcx-scrapi` library

!pip install dfcx-scrapi

# Imports


In [ ]:
from dfcx_scrapi.tools.copy_util import CopyUtil

# User Inputs
In the next section, we will collect runtime variables needed to execute this notebook.   
This should be the only cell of the notebook you need to edit in order for this notebook to run.

For this notebook, we'll need the following inputs:

- `creds_path`: Your local path to your GCP Service Account Credentials
- `agent_id`: Your Dialogflow CX Agent ID in String format

In [ ]:
creds_path = '<YOUR_CREDS_FILE>'
agent_id = '<YOUR_AGENT_ID>'
source_flow = 'Default Start Flow'
target_flow = 'My Target Flow'

# Get Flows Map from Agent
First, we will extract a `map` of the Flow IDs and Flow Display Names from the Agent.   
We pass `reverse=True` to the `get_flows_map` function which provides the Display Names as keys and the IDs as values.   
This allows us to refernce the map using Display Names vs. the long, cumbersome ID names.

In [ ]:
cu = CopyUtil(creds_path)

flows_map = cu.flows.get_flows_map(agent_id, reverse=True)

# Get All Pages from Source Flow
Once we have our `flows_map`, we will use it to extract all of the Page objects in our `Source` Flow.   
In our case, we want to extract pages from the Flow named `Default Start Flow`.  
We will call the `list_pages` function and pass in the appropriate dictionary reference.

In [16]:
my_pages = cu.pages.list_pages(flows_map[source_flow])
print('{} Page Count = {}'.format(source_flow, len(my_pages)))

Default Start Flow Page Count = 3


# Extract Subset of Pages To Copy
Now that we have `my_pages` (a List of Page objects) we need to extract the subset of pages that we plan on copying over to our `Target` Flow.   
Usually you will do this using some regex matcher or pattern to select your pages.  
The easiest way to do this is to ensure the the Page designer has prepended the `page.display_name` with a specific label.    
The more unique the matching pattern, the better!

Ex:
- MyFlow - Page 1
- MyFlow - Page 2
- MyFlow - Page N

In [17]:
subset_pages = [] # define a list placeholder for your Page proto objects
for page in my_pages:
    if 'MyFlow -' in page.display_name: # Comment out this line to copy the whole flow
        subset_pages.append(page)

print('Total Pages to Copy = {}'.format(len(subset_pages)))

Total Pages to Copy = 2


# Create Page Shells in Target Flow
Using the `subset_pages` that we just collected, we will iterate through them and create a "shell" page in the `Target` flow.  
This allows CX to assign a new UUID for the `Target` page which we will use to replace all references in the existing `subset_pages`

Remember to pass in the `Target Flow ID` using the reverse flows map from above.

**NOTE** - If you have a lot of pages, consider a `time.sleep(.5)` in your loop so as to not overrun your API limits!

Note that this only needs to be run once. 
Once the empty pages have been created in a given flow, they cannot be created again.
Trying to create them again will give an AlreadyExists error.

In [18]:
for page in subset_pages:
    cu.pages.create_page(flows_map[target_flow], display_name=page.display_name)

# Copy Route Groups to Target Flow

In [ ]:
cu.copy_route_groups(agent_id, source_flow, target_flow)

# Modify Page Objects
In a previous step, we collected the `subset_pages` list of Page objects.  
Now we will use a 2-step process comprised of the following ~!* MAGIC *!~ `CopyUtil` functions:
1. `convert_from_source_page_dependencies`
2. `convert_to_destination_page_dependencies`

In Step #1, we will use the following args:
- `agent_id`
- `subset_pages` (i.e. the original List of Page Objects we collected)
- `source_flow`

This will modify all of the UUIDs in the Page objects to be _PLAIN TEXT STRING DISPLAY NAMES_ using internal map functions.  
We will store the results in a variable called `subset_pages_prepped`

In Step #2, we will use the following args:
- `agent_id`
- `subset_pages_prepped` (i.e. our _MODIFIED_ List of Page Objects we collected)
- `target_flow`

This will perform a reverse dictionary lookup on all of the previously modified resources using internal map functions and give them their newly assigned UUIDs.   
We will store the results in a variable called `final_pages`

In [19]:
# Step 1
subset_pages_prepped = cu.convert_from_source_page_dependencies(agent_id, subset_pages, source_flow)

# Step 2
final_pages = cu.convert_to_destination_page_dependencies(agent_id, subset_pages_prepped, target_flow)

# Update Pages in Target Flow
Our final step is to loop through our `final_pages` list and call the `update_page` function for each Page in the list.   
This will write the modified Page objects to our Dialogflow CX Agent.

In [20]:
for page in final_pages:
    cu.pages.update_page(page.name, page)
    print('Updated Page: {}'.format(page.display_name))

Updated Page: MyFlow - Test2
Updated Page: MyFlow - Test1


# Update Start Page in Target Flow

Since the Start Page of a given flow is not a Page object in CX, but rather a Flow object, the above will not copy over the contents of the Start Page. If you are copying an entire flow into another flow, and need to copy the Start Page as well, the `convert_start_page_dependencies` function can be used to copy the Start Page contents.

In [ ]:
# Get the source and target flow objects
source_flow_obj = cu.flows.get_flow_by_display_name(source_flow, agent_id)
target_flow_obj = cu.flows.get_flow_by_display_name(target_flow, agent_id)

# Convert resources of start page in source flow to update start page in target flow
converted_source = cu.convert_start_page_dependencies(
    agent_id, source_flow_obj, agent_type='source', flow=source_flow
)
converted_target = cu.convert_start_page_dependencies(
    agent_id, converted_source, agent_type='destination', flow=target_flow
)

# Update start page in target flow
cu.flows.update_flow(converted_target.name, converted_target)
print('Updated Flow: {}'.format(target_flow))

# Final Thoughts and Wrap-Up
All of the above could be combined into a single function to streamline the process.  
However, for instructional purposes, breaking the workflow up into parts allows the user to understand the context of each function and its core purpose.   

In addition, other modifications not covered in this tutorial could be applied inline prior to pushing the Page updates in the final step.   
As noted at the start, this notebook can also be modified to copy pages _BETWEEN_ agents as well.